In [24]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [ ]:
 # Create a connection to the PostgreSQL database
pg_conn = psycopg2.connect(
    host="34.93.93.62",
    database="vipani_devpy_final_stg",
    user="postgres2",
    password="your_password_here",
    port="5432"
)
pg_cur = pg_conn.cursor()

# SQL query to fetch data from the order_lineitems_hive table
pg_cur.execute("""
    SELECT 
        id,
        rfq_id,
        seller_id,
        status,
        cdate as time_stamp
    FROM bt_rfq_quotes
""")

rfq_data = pg_cur.fetchall()
rfq_df = pd.DataFrame(rfq_data, columns=['id', 'rfq_id', 'seller_id', 'status', 'time_stamp'])
# Close the cursor and connection
pg_cur.close()

In [26]:
rfq_df

,id,rfq_id,seller_id,status,time_stamp
0,2780,3783,509,6,2025-06-12 07:24:07.826890+00:00
1,2803,3824,509,6,2025-06-19 08:47:26.298384+00:00
2,2781,3788,91,14,2025-06-11 09:15:54.780439+00:00
3,2783,3788,91,11,2025-06-13 06:38:03.692332+00:00
4,2784,3794,509,6,2025-06-13 10:15:23.760788+00:00
...,...,...,...,...,...
3157,2779,3783,509,11,2025-06-12 07:23:48.085903+00:00
3158,2775,3784,509,5,2025-06-12 06:39:19.660718+00:00
3159,2773,3783,509,5,2025-06-12 06:18:24.374564+00:00
3160,2774,3783,509,5,2025-06-12 06:21:59.881460+00:00


In [27]:
df_latest = (
    rfq_df.sort_values('time_stamp')
      .drop_duplicates(subset=['rfq_id', 'seller_id'], keep='last')
)


In [28]:
df_latest

,id,rfq_id,seller_id,status,time_stamp
407,1,13,509,5,2024-11-21 09:38:58.976525+00:00
406,2,16,509,5,2024-11-21 10:30:17.572407+00:00
408,3,17,509,10,2024-11-21 10:42:35.021653+00:00
410,5,18,509,10,2024-11-21 12:42:03.460623+00:00
411,6,22,509,5,2024-11-21 12:50:59.039463+00:00
...,...,...,...,...,...
385,3160,4440,509,9,2025-09-08 08:38:53.030779+00:00
386,3162,4442,91,6,2025-09-08 09:26:28.177820+00:00
365,3163,4441,91,5,2025-09-08 09:52:29.768920+00:00
388,3165,4445,93,6,2025-09-08 12:06:03.503981+00:00


In [29]:
df_latest['status'].unique()

array([ 5, 10, 12,  9, 11,  6, 14, 18,  3, 15, 19, 22, 20, 24])

In [30]:
result = (
    df_latest.groupby('seller_id')
    .agg(
        total_id1=('rfq_id', 'count'),
        active_id1=('status', lambda x: x.isin([14, 15]).sum())
    )
    .reset_index()
)


In [31]:
result

,seller_id,total_id1,active_id1
0,22,1,0
1,54,1,0
2,91,1091,59
3,92,4,2
4,93,51,5
5,94,3,0
6,103,3,0
7,130,1,0
8,136,1,0
9,181,3,0
